# __From raw twitter scrape json to clean fine tune babyfood jsonl__

## snatch tweets from json
reads a file that is a list of dictionaries where the target (tweet) is the value with a key named "text"

In [3]:
import json
 
# Opening JSON file
f = open('twitter_dump2.json')
json_file = json.load(f)

text_file = []

for dictionary in json_file:
    text_file.append(dictionary["text"])
    

## tweet cleaner
if we decide hashtags, emojis or @mentions ae a part of contemporary expressions that belong into poetry we could comment out

In [4]:
import re
import emoji


def tweet_cleaner(tweet):
    # remove mentions
    mentions_free = re.sub("@\w+", "", tweet)
    # remove hashtags
    hashtag_free = re.sub("#\w+", "", mentions_free)
    # special shortcuts
    rt_free = re.sub("^RT", "", hashtag_free)
    # remove url 
    url_free = re.sub(r'http\S+', '', rt_free)
    # remove emojis
    emo_free = emoji.get_emoji_regexp().sub(u'', url_free)
    # remove white space
    stripped = emo_free.strip()
  
    return stripped

In [5]:
clean_tweets = [tweet_cleaner(x)for x in text_file]
len(clean_tweets)

73837

## unicode
this removes stuff like \xa \n and so on

In [6]:
def unicode_maker(text):
    # creating a unicode string
    text_encode = text.encode(encoding="ascii", errors="ignore")
    text_decode = text_encode.decode()
    clean_text = " ".join([word for word in text_decode.split()])
    return clean_text

uni_tweets = [unicode_maker(x)for x in clean_tweets]
uni_tweets

[': Deployment confirmed, s DART is on its way to redirect an asteroid',
 ': Watch Falcon 9 launch s DART mission humanitys first planetary defense test to redirect an asteroid',
 'About to launch asteroid defense test!',
 'That is the great question',
 'Exactly',
 '',
 ': All systems and weather are looking good for tonights Falcon 9 launch of s DART into an asteroid-intercepting interplane',
 'Exactly (sigh)',
 'Sounds good',
 'It can simulate anywhere, but we need to focus on one region until FSD is out of beta, then expand geographically to have betas in other countries. Canada is not a lot different from US, but different enough.',
 'High-performance code is in C using our own compiler to maximize frame rate. We use Python with custom libraries for fast prototyping, then convert to C for fast execution. New software is run against real-world video &amp; our world simulator to evaluate performance, then QA.',
 'I dont use Binance (tried at one point, but signup was too many hoops t

## remove punctuation, numbers etc.

In [7]:
import string

In [8]:
def number_punct(sentence):
    # remove numbers
    number_free = ''.join(word for word in sentence if not word.isdigit())
    # remove punctuation
    punctuation_free = "".join(
        [i for i in number_free if i not in string.punctuation])
    # lower case
    upper_free = punctuation_free.lower()
    
    return upper_free

number_free = [number_punct(x) for x in uni_tweets]
number_free

[' deployment confirmed s dart is on its way to redirect an asteroid',
 ' watch falcon  launch s dart mission humanitys first planetary defense test to redirect an asteroid',
 'about to launch asteroid defense test',
 'that is the great question',
 'exactly',
 '',
 ' all systems and weather are looking good for tonights falcon  launch of s dart into an asteroidintercepting interplane',
 'exactly sigh',
 'sounds good',
 'it can simulate anywhere but we need to focus on one region until fsd is out of beta then expand geographically to have betas in other countries canada is not a lot different from us but different enough',
 'highperformance code is in c using our own compiler to maximize frame rate we use python with custom libraries for fast prototyping then convert to c for fast execution new software is run against realworld video amp our world simulator to evaluate performance then qa',
 'i dont use binance tried at one point but signup was too many hoops to jump through so no incon

In [9]:
## removes tweets with less than two words
more_than_two = [x for x in number_free if (len(x.split())>2)]

## removes duplicates
clean_set = set(more_than_two)
clean_set

{'donde esta juan pablo',
 ' love all of this especially this note about women at work we care deeply and we bring our lived experience to our work f',
 'i love having all the bullshit of my life cut out all at once i cant believe how much accumulated i hate it now i have to think about all the stuff i usually stay too busy to think about',
 'congratulations on this welldeserved recognition like you im optimistic that we will be able to bring this pandemic to an end your work with the and many others will also help us to better prepare for future pandemics',
 'here are four other ways that america can advance its leadership on climate change this year and put the world on a path to zero emissions by ',
 'juneteenth is a day of profound weight and profound power',
 'and world inequality lab newsletter march  welcome to the fourth and world inequality lab newsletter which covers our main developments since october  plenty has happened over this period',
 'what ive been saying for a while

## remove empty strings and excess whitespaces
strings become empty when they where just made of mentions, url, hashtags etc.

In [10]:
# remove empty strings
no_empty_strings = [i for i in clean_set if i]
# remove outer whitespace 
stripped = [x.strip() for x in no_empty_strings]
# remove inner whitespace 
one_space =  [re.sub(' +', ' ', x) for x in stripped]
one_space

['donde esta juan pablo',
 'love all of this especially this note about women at work we care deeply and we bring our lived experience to our work f',
 'i love having all the bullshit of my life cut out all at once i cant believe how much accumulated i hate it now i have to think about all the stuff i usually stay too busy to think about',
 'congratulations on this welldeserved recognition like you im optimistic that we will be able to bring this pandemic to an end your work with the and many others will also help us to better prepare for future pandemics',
 'here are four other ways that america can advance its leadership on climate change this year and put the world on a path to zero emissions by',
 'juneteenth is a day of profound weight and profound power',
 'and world inequality lab newsletter march welcome to the fourth and world inequality lab newsletter which covers our main developments since october plenty has happened over this period',
 'what ive been saying for a while the

## add whitespace in the beginning for openai's 
tokenization and stopping . END

In [11]:
start_end = [f" {x}. END" for x in stripped] 
start_end

[' donde esta juan pablo. END',
 ' love all of this especially this note about women at work we care deeply and we bring our lived experience to our work f. END',
 ' i love having all the bullshit of my life cut out all at once i cant believe how much accumulated i hate it now i have to think about all the stuff i usually stay too busy to think about. END',
 ' congratulations on this welldeserved recognition like you im optimistic that we will be able to bring this pandemic to an end your work with the and many others will also help us to better prepare for future pandemics. END',
 ' here are four other ways that america can advance its leadership on climate change this year and put the world on a path to zero emissions by. END',
 ' juneteenth is a day of profound weight and profound power. END',
 ' and world inequality lab newsletter march  welcome to the fourth and world inequality lab newsletter which covers our main developments since october  plenty has happened over this period. 

## Creates a DataFrame with a prompt and completion column

In [12]:
import pandas as pd

In [13]:
# keyword = json_file[0]["segment"]
df_prompt_comp = pd.DataFrame({"prompt": "", "completion":start_end})
df_dub_free = df_prompt_comp.drop_duplicates()

__saves json babyfood__

In [14]:
df_dub_free.to_json("musician1.json", orient = "records", lines= True)

# __from song parts to babyfood__
saves some song examples in a txt file and get json with linebreaks and end bit (END) starting whitespace - songparts are devided by blank line (\n\n respectively) 

In [197]:
import re

In [197]:
# load txt file with song examples - examples are divided by whiteline
rap_file = list(open("eight_bars.txt", "r"))
rap_list = "".join(x for x in rap_file)

# split examples by whiteline \n\n
chunks = re.split('\n\n', rap_list)

# replace \' by '
apo = [sub.replace("\'", "'") for sub in chunks]

# remove empty strings
no_empty_rap = [i for i in apo if i]

# put leading whitespace and END bit for better tokenization and GTP3 learning
space_rap = [f" {x} END" for x in no_empty_rap] 

# remove unicode
string_decode =  [x.encode("ascii", "ignore").decode() for x in space_rap] 

# make a pandas Dataframe
df_rap = pd.DataFrame({"prompt": "...->", "completion":string_decode})

# save into a GTP3 learnable jsonl with linebreaks and END bit to give as a example document 
df_rap.to_json("rap_doc5.json", orient = "records", lines= True)

# Not Used

creates text file without prompts just clean tweets line by line

In [65]:
one_long_string = ".\n".join(one_space)

with open("file10.txt", "w") as output:
    output.write(one_long_string)

## Tokenize
spits out a list of lists of important words 

In [206]:
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))


def text_to_token(sentence):
    # tokenize
    word_tokens = word_tokenize(sentence) 
    # remove stopwords
    important_words = [w for w in word_tokens if not w in stop_words] 
    
    return important_words

In [207]:
tokens = [text_to_token(s) for s in one_space]
tokens

[['universe',
  'works',
  'computers',
  'use',
  'video',
  'games',
  'learn',
  'function',
  'physical',
  'world'],
 ['seriously',
  'guys',
  'actually',
  'besides',
  'act',
  'outraged',
  'press',
  'know',
  'rarely',
  'see',
  'constituents'],
 ['ive', 'got', 'go', 'back', 'watch', 'parts'],
 ['transformation',
  'rwandas',
  'health',
  'system',
  'roots',
  'simple',
  'powerful',
  'belief',
  'everyone',
  'deserves',
  'shot',
  'healthy',
  'life',
  'matter',
  'income',
  'ethnic',
  'group',
  'call',
  'home'],
 ['sometimes', 'socalled', 'law', 'order', 'timeswellnot', 'much'],
 ['weekwhy', 'dont', 'get', 'town'],
 ['three',
  'days',
  'conference',
  'hosted',
  'collaboration',
  'w',
  'visit',
  'htt'],
 ['dont', 'play', 'allstar', 'game', 'nba', 'wont', 'let'],
 ['whew', 'put', 'foot', 'one'],
 ['watching',
  'oscar',
  'short',
  'subject',
  'docs',
  'films',
  'stands',
  'top',
  'immediately',
  'relevant'],
 ['great',
  'speaking',
  'advisory',
  

## Lemmatize
gets rid of plurals aso. without mutliating words like stemming.

In [208]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

def get_lemm(l):
    lemm = [lemma.lemmatize(word) for word in l]
    return lemm

In [209]:
lemmas = [get_lemm(s) for s in tokens]
lemmas

[['universe',
  'work',
  'computer',
  'use',
  'video',
  'game',
  'learn',
  'function',
  'physical',
  'world'],
 ['seriously',
  'guy',
  'actually',
  'besides',
  'act',
  'outraged',
  'press',
  'know',
  'rarely',
  'see',
  'constituent'],
 ['ive', 'got', 'go', 'back', 'watch', 'part'],
 ['transformation',
  'rwanda',
  'health',
  'system',
  'root',
  'simple',
  'powerful',
  'belief',
  'everyone',
  'deserves',
  'shot',
  'healthy',
  'life',
  'matter',
  'income',
  'ethnic',
  'group',
  'call',
  'home'],
 ['sometimes', 'socalled', 'law', 'order', 'timeswellnot', 'much'],
 ['weekwhy', 'dont', 'get', 'town'],
 ['three',
  'day',
  'conference',
  'hosted',
  'collaboration',
  'w',
  'visit',
  'htt'],
 ['dont', 'play', 'allstar', 'game', 'nba', 'wont', 'let'],
 ['whew', 'put', 'foot', 'one'],
 ['watching',
  'oscar',
  'short',
  'subject',
  'doc',
  'film',
  'stand',
  'top',
  'immediately',
  'relevant'],
 ['great',
  'speaking',
  'advisory',
  'council',
 